In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import networkx.algorithms.isomorphism as iso
import sympy
import random
import time
import itertools
import math
from IPython.display import clear_output

In [2]:
def graphlet_list(N):
    assert N > 0
    foo = 1
    loc_graphlet_list = {n: [] for n in range(1,N+1)}
    while True:
        G = nx.graph_atlas(foo)
        n = G.number_of_nodes()
        if n>N:
            break
        if nx.is_connected(G):
            loc_graphlet_list[n].append(G)
        foo += 1
    return loc_graphlet_list
    

def find_type_match(T):
    n = T.number_of_nodes()
    if n==1:
        return((0, {u: 0 for u in T.nodes()}))
    if n==2:
        return((0, {u: i for i,u in enumerate(T.nodes())}))
    if n==3:
        if T.number_of_edges()==2:
            u0 = next((node for node in T.nodes() if T.degree(node)==2))
            (u1,u2) = (node for node in T.neighbors(u0))
            return((0, {u0: 0, u1: 1, u2: 2}))
        if T.number_of_edges()==3:
            return((1,{u:i for i,u in enumerate(T.nodes())}))
    if n==4:
        e_num = T.number_of_edges()
        max_degree = max((T.degree(node) for node in T.nodes()))
        if e_num==3 and max_degree==3:
            u3 = next((node for node in T.nodes() if T.degree(node)==3))
            (u0,u1,u2) = (node for node in T.neighbors(u3))
            return((0, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==3 and max_degree==2:
            (u0,u1) = (node for node in T.nodes() if T.degree(node)==2)
            u2 = next((node for node in T.neighbors(u1) if node!=u0))
            u3 = next((node for node in T.neighbors(u0) if node!=u1))
            return((1, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==4 and max_degree==3:
            u3 = next((node for node in T.nodes() if T.degree(node)==3))
            (u1,u2) = (node for node in T.nodes() if T.degree(node)==2)
            u0 = next((node for node in T.nodes() if T.degree(node)==1))
            return((2, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==4 and max_degree==2:
            u0 = next((node for node in T.nodes()))
            (u1,u3) = (node for node in T.neighbors(u0))
            u2 = next((node for node in T.neighbors(u1) if node!=u0))
            return((3, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==5:
            (u0,u2) = (node for node in T.nodes() if T.degree(node)==3)
            (u1,u3) = (node for node in T.nodes() if T.degree(node)==2)
            return((4, {u0:0, u1:1, u2:2, u3:3}))
        if e_num==6:
            (u0,u1,u2,u3) = (node for node in T.nodes())
            return((5, {u0:0, u1:1, u2:2, u3:3}))
    # Improve matching procedure here for n>4.
    GM = next((i, iso.GraphMatcher(T,T_)) 
              for (i,T_) in enumerate(cached_graphlet_list[n]) 
              if iso.GraphMatcher(T,T_).is_isomorphic())
    assert GM[1].is_isomorphic()
    return((GM[0],GM[1].mapping))

def find_type(T):
    n = T.number_of_nodes()
    if n==1:
        return 0
    if n==2:
        return 0
    if n==3:
        if T.number_of_edges()==2:
            return 0
        if T.number_of_edges()==3:
            return 1
    if n==4:
        e_num = T.number_of_edges()
        max_degree = max((T.degree(node) for node in T.nodes()))
        if e_num==3 and max_degree==3:
            return 0
        if e_num==3 and max_degree==2:
            return 1
        if e_num==4 and max_degree==3:
            return 2
        if e_num==4 and max_degree==2:
            return 3
        if e_num==5:
            return 4
        if e_num==6:
            return 5
    # Improve matching procedure here at least for n=4.
    GM = next((i 
              for (i,T_) in enumerate(cached_graphlet_list[n]) 
              if iso.GraphMatcher(T,T_).is_isomorphic()))
    return GM  

def prob_functions(N):
    assert N > 0
    x = {0: sympy.var('x_0')}
    y = {0: sympy.var('y_0')}
    prob = {1: {0: x[0]/2}}
    if N > 1:
        x[1] = sympy.var('x_1')        
        y[1] = sympy.var('y_1')
        prob[2] = {0: sympy.Integer(1)}
    for n in range(3, N+1):
        x[n-1] = sympy.var('x_{}'.format(n-1))
        y[n-1] = sympy.var('y_{}'.format(n-1))
        prob[n] = {}
        for T_ind, T in enumerate(cached_graphlet_list[n]):
            prob[n][T_ind] = 0
            for u in T.nodes():
                S = subgraph(T, T.nodes()-{u})
                if not nx.is_connected(S):
                    continue
                S_ind, S_match = find_type_match(S)
                S_prob = (prob[n-1][S_ind]
                          .subs({x[i]:y[i] for i in range(n-1)})
                          .subs({y[j]:x[i] for i,j in S_match.items()})
                         )
                S_deg = sum(x[i] for i in S.nodes()) - 2*S.number_of_edges()
                prob[n][T_ind] += S_prob * T.degree(u) / S_deg                 
    return prob[N]

def subgraph(G, nodes):
    list_nodes = list(nodes)
    T = nx.Graph()
    T.add_nodes_from(nodes)
    for i in range(len(nodes)):
        for j in range(i):
            if list_nodes[i] in G.neighbors(list_nodes[j]):
                T.add_edge(list_nodes[i],list_nodes[j])
    return T


In [3]:
N=4
x = [sympy.var('x_{}'.format(i)) for i in range(N)]
cached_graphlet_list = graphlet_list(N)
cached_prob = {ind: sympy.lambdify(x, func) 
                    for ind, func in prob_functions(N).items()
              }
#4s for N=5, 2m43s for N=6

In [296]:
def lift(G, vert, k):
    graphlet = set([vert])
    if k==1:
        return graphlet
    u = vert
    neig_list = []
    for n in range(2, k+1):
        neig_list = ([v for v in neig_list if v!=u] 
                     + [v for v in G.neighbors(u) if v not in graphlet])
        u = random.choice(neig_list)
        graphlet.add(u)
    return graphlet

def random_walk_nodes(G, v0, steps_num):
    curr_vert = v0
    for _ in range(steps_num):
        curr_vert = random.choice(list(G.neighbors(curr_vert)))
    return curr_vert

def liftSRW(G, N, time_limit=None, query_limit=None, epoch_num=1, time_step=10, 
            output_form='count', ground_truth=None, steps_mult=None, steps_num=None):

    assert (time_limit is None) != (query_limit is None)
    assert (steps_mult is None) != (steps_num is None)
    
    if steps_mult is not None:
        steps_between_lifts = int(steps_mult*math.log(G.number_of_nodes()))
    else:
        steps_between_lifts = steps_num
    norm_error = 0
    
    for epoch in range(epoch_num):

        #print('Starting epoch {}'.format(epoch+1))
        type_counter = {i:0 for i in range(len(cached_graphlet_list[N]))}
        t0 = time.time()
        lift_count = 0
        query_count = 0
        time_iter_count = 1
        stop_condition = False
        v = random.choice(list(G.nodes()))
        flag = 0

        while not stop_condition:
            v = random_walk_nodes(G, v, steps_between_lifts)
            T = lift(G, v, N)
            T_type, T_match = find_type_match(subgraph(G,T))
            inv_match = {i: j for j,i in T_match.items()}
            degree_list = [G.degree(inv_match[i]) for i in range(N)]
            T_prob = cached_prob[T_type](*degree_list)
            type_counter[T_type] += (T_prob)**(-1)
            lift_count += 1
            curr_time = time.time()
            
            if curr_time - t0 > time_iter_count*time_step:
                if output_form=='count':
                    print("Time is {} Type counter is {}"
                          .format(int(curr_time-t0), 
                                  scale(type_counter, 
                                        G.number_of_edges()*lift_count**(-1))))
                if output_form=='ratio':
                    print("Time is {} Type ratios are {}"
                          .format(int(curr_time-t0), 
                                  normalize(type_counter)))
                print("Time is {} NMSE is {}"
                      .format(int(curr_time-t0), 
                              NMSE(type_counter, ground_truth)))
                print("Number of graphlets sampled is {}".format(lift_count))
                time_iter_count += 1
                
            if time_limit is not None:
                stop_condition = (time.time()-t0 > time_limit)
            if query_limit is not None:
                query_count += steps_between_lifts + N - 1
                stop_condition = (query_count > query_limit)


        if ground_truth is not None:
            error = NMSE(type_counter, ground_truth)
            #print("NMSE error is {}".format(error))
            norm_error += error
        #print("")
    print("Number of lifts is {}".format(lift_count))
    norm_error = norm_error*(epoch_num)**(-1)
    return {'count': scale(type_counter, G.number_of_edges()*lift_count**(-1)), 
            'NMSE': norm_error}

def brute_force(G, N=3):
    type_counter = {i:0 for i in range(len(cached_graphlet_list[N]))}
    if N==3:
        percent_count = 0
        counter = 0
        for u,v in G.edges():
            for w in set(G.neighbors(u))-{v}:
                if w in G.neighbors(v): 
                    type_counter[1] += 1
                else:
                    type_counter[0] += 1
            for w in set(G.neighbors(v))-{u}:
                if w in G.neighbors(u): 
                    type_counter[1] += 1
                else:
                    type_counter[0] += 1
            counter += 1
            if counter > percent_count*0.00001*G.number_of_edges():
                clear_output()
                #print("{}% complete".format(percent_count*0.001))
                percent_count += 1
        type_counter[0] = type_counter[0]/2
        type_counter[1] = type_counter[1]/6
        
    if N==4:
        for u,v in G.edges():
            neigh = set(G.neighbors(u)).union(set(G.neighbors(v)))-{u,v}
            for w,z in itertools.combinations(neigh, 2):
                T = subgraph(G, {u,v,w,z})
                type_counter[find_type(T)] += 1
        type_counter[0] = type_counter[0]/3
        type_counter[1] = type_counter[1]
        type_counter[2] = type_counter[2]/3
        type_counter[3] = type_counter[3]/4
        type_counter[4] = type_counter[4]/5
        type_counter[5] = type_counter[5]/6
    return type_counter

def NMSE(dict_hat, dict_true):
    norm_dict_hat = normalize(dict_hat)
    norm_dict_true = normalize(dict_true)
    return sum(((norm_dict_hat[i]*freq**(-1) - 1)**2
                for i, freq in norm_dict_true.items() if freq != 0))

def normalize(dict_hat):
    total_count = sum((val for i,val in dict_hat.items()))
    return {i: val*(total_count)**(-1) for i,val in dict_hat.items()}

def scale(dict_hat, scalar):
    return {i: int(val*scalar) for i,val in dict_hat.items()}

def load_graph(name, N):

    if name=='com-amazon':
        G = nx.read_edgelist(
            'Graphs/com-amazon.ungraph.txt',
            create_using = nx.Graph())
        
        if N==3:
            return {
                'graph': G,
                'ground_truth': {0: 7750799, 
                                 1: 667129}
            }
        if N==4:
            return {
                'graph': G,
                'ground_truth':  {0: 124295537, 
                                  1: 37383434, 
                                  2: 13674662, 
                                  3: 422515, 
                                  4: 1874925, 
                                  5: 275961}
            }
        if N>4:
            return {
                'graph': G,
                'ground_truth':  None
            }
        

    if name=='com-dblp':
        G = nx.read_edgelist(
            'Graphs/com-dblp.ungraph.txt',
            create_using = nx.Graph())
        if N==3:
            return {
                'graph': G,
                'ground_truth':  {0: 15107734, 
                                  1: 2224385}
            }
        if N==4:
            return {
                'graph': G,
                'ground_truth':  {0: 258570802, 
                                  1: 252447350, 
                                  2: 96615211, 
                                  3: 203394, 
                                  4: 4764685, 
                                  5: 16713192}
            }
        if N>4:
            return {
                'graph': G,
                'ground_truth':  None
            }

    if name=='com-lj':
        G = nx.read_edgelist(
            'Graphs/com-lj.ungraph.txt',
            create_using = nx.Graph())
        if N==3:
            return {
                'graph': G,
                'ground_truth':  {0: 3722307805, 
                                  1: 177820130}
            }
        if N==4:
            return {
                'graph': G,
                'ground_truth':  {0: 1983908933796,
                                  1: 542683013686,
                                  2: 57662704306,
                                  3: 2541452010,
                                  4: 8190586835,
                                  5: 5216918441}
            }
        if N>4:
            return {
                'graph': G,
                'ground_truth':  None
            }

    if name=='com-youbube':
        G = nx.read_edgelist(
            'Graphs/com-youtube.ungraph.txt',
            create_using = nx.Graph())
        if N==3:
            return {
                'graph': G,
                'ground_truth':  {0: 1465313402, 
                                  1: 3056386}
            }
        if N==4:
            return {
                'graph': G,
                'ground_truth':  {0: 5730407268993,
                                  1: 91488735459,
                                  2: 12371157628,
                                  3: 231979854,
                                  4: 221833272,
                                  5: 4986965}
            }
        if N>4:
            return {
                'graph': G,
                'ground_truth':  None
            }

    if name=='misc-net25':
        G = nx.read_edgelist(
            'Graphs/misc-net25.mtx',
            create_using = nx.Graph())
        for v in G.nodes():
            G.remove_edge(v,v)
        if N==3:
            return {
                'graph': G,
                'ground_truth':  {0: 12690840, 
                                  1: 64090}
            }
        if N==4:
            return {
                'graph': G,
                'ground_truth':  {0: 361490550,
                                  1: 550792350,
                                  2: 12554670,
                                  3: 44915955,
                                  4: 0,
                                  5: 0}
            }
        if N>4:
            return {
                'graph': G,
                'ground_truth':  None
            }

    if name=='misc-as-caida':
        G = nx.read_edgelist(
            'Graphs/misc-as-caida.mtx',
            create_using = nx.Graph())
        for v in G.nodes():
            G.remove_edge(v,v)    
        if N==3:
            return {
                'graph': G,
                'ground_truth':  {0: 59513652, 
                                  1: 72730}
            }
        if N==4:
            return {
                'graph': G,
                'ground_truth':  {0: 62565214368,
                                  1: 2808802860,
                                  2: 203097552,
                                  3: 3774144,
                                  4: 4084544,
                                  5: 0}
            }
        if N>4:
            return {
                'graph': G,
                'ground_truth':  None
            }

    if name=='misc-fullb':
        G = nx.read_edgelist(
            'Graphs/misc-fullb.mtx',
            create_using = nx.Graph())
        for v in G.nodes():
            G.remove_edge(v,v)
        if N==3:
            return {
                'graph': G,
                'ground_truth':  {0: 162067420, 
                                  1: 60212260}
            }
        if N==4:
            return {
                'graph': G,
                'ground_truth':  {0: 1078734774,
                                  1: 4837795036,
                                  2: 2707584768,
                                  3: 64898820,
                                  4: 897215295,
                                  5: 370980150}
            }
        if N>4:
            return {
                'graph': G,
                'ground_truth':  None
            }

    if name=='misc-neos3':
        G = nx.read_edgelist(
            'Graphs/misc-neos3.mtx',
            create_using = nx.Graph())
        for v in G.nodes():
            G.remove_edge(v,v)
        if N==3:
            return {
                'graph': G,
                'ground_truth':  {0: 207426691, 
                                  1: 505603}
            }
        if N==4:
            return {
                'graph': G,
                'ground_truth':  {0: 59618248397,
                                  1: 11164704825,
                                  2: 120388385,
                                  3: 2047846,
                                  4: 499122,
                                  5: 0}
            }
        if N>4:
            return {
                'graph': G,
                'ground_truth':  None
            }
        

    if name=='misc-discogs_affiliation':
        G = nx.read_edgelist(
            'Graphs/misc-discogs_affiliation.edges',
            create_using = nx.Graph())
        for v in G.nodes():
            G.remove_edge(v,v)
        if N==3:
            return {
                'graph': G,
                'ground_truth':  None
            }
        if N==4:
            return {
                'graph': G,
                'ground_truth':  {0: 208345722513295,
                                  1: 851118877585,
                                  2: 58223406336,
                                  3: 3008868833,
                                  4: 439215089,
                                  5: 654413}
            }
        if N>4:
            return {
                'graph': G,
                'ground_truth':  None
            }

    if name=='misc-amazon-ratings':
        G = nx.read_edgelist(
            'Graphs/misc-amazon-ratings.edges',
            create_using = nx.Graph())
        for v in G.nodes():
            G.remove_edge(v,v)
        if N==3:
            return {
                'graph': G,
                'ground_truth':  {0: 699425719, 
                                  1: 79638}
            }
        if N==4:
            return {
                'graph': G,
                'ground_truth':  {0: 719668204837,
                                  1: 40966346985,
                                  2: 184396006,
                                  3: 37045086,
                                  4: 561566,
                                  5: 671}
            }
        if N>4:
            return {
                'graph': G,
                'ground_truth':  None
            }

    if name=='misc-dbpedia-all':
        G = nx.read_edgelist(
            'Graphs/misc-dbpedia-all.edges',
            create_using = nx.Graph())
        for v in G.nodes():
            G.remove_edge(v,v)
        if N==3:
            return {
                'graph': G,
                'ground_truth':  {0: 174250340949, 
                                  1: 8329548}
            }
        if N==4:
            return {
                'graph': G,
                'ground_truth':  {0: 19646604300441472,
                                  1: 1652259549599,
                                  2: 622928133900,
                                  3: 15925209557,
                                  4: 15630164176,
                                  5: 4609834}
            }
        if N>4:
            return {
                'graph': G,
                'ground_truth':  None
            }

In [ ]:
G, ground_truth = [load_graph('com-lj',4)[key] for key in ['graph','ground_truth']]
print(ground_truth)
print(math.log(G.number_of_nodes()))

In [ ]:
epoch_num = 200
for num in range(1,9):
    print("Start testing step_num={}".format(num))
    print("NMSE is",liftSRW(G, N, epoch_num=epoch_num, query_limit=5000, time_step=10, 
                  steps_num=num,
                  ground_truth=ground_truth)['NMSE'])

In [237]:
ground_truth = {0: 1238448, 1: 420329}

epoch_num = 40
alpha_array = [0.2*(ind+1) for ind in range(5)]
error_array = []
for alpha in alpha_array:
    print("Test alpha equal to {0:.1f}".format(alpha))
    error_array.append(liftSRW(G2, N, 
                               epoch_num=epoch_num, 
                               time_limit=5, time_step=30, 
                               steps_mult=alpha, 
                               ground_truth=ground_truth)['NMSE'])
    print("For alpha equal to {0:.1f} error is {1:.5f}".format(alpha,error_array[-1]))
    print(" ")
plt.plot(alpha_array,error_array)
plt.show()

Test alpha equal to 0.2
Number of lifts is 25311
NMSE error is 1.23188297554e-06
Number of lifts is 24470
NMSE error is 0.000157850158755
Number of lifts is 24066
NMSE error is 0.000192320769519
Number of lifts is 23665
NMSE error is 0.000101650862583
Number of lifts is 24307
NMSE error is 1.05987752145e-05
Number of lifts is 23098
NMSE error is 5.04512277051e-09
Number of lifts is 23810
NMSE error is 3.89590110831e-06
Number of lifts is 21229
NMSE error is 0.000159164779287
Number of lifts is 25099
NMSE error is 0.000108122490108
Number of lifts is 24259
NMSE error is 0.000375347557346
Number of lifts is 22855
NMSE error is 0.000253532780239
Number of lifts is 24440
NMSE error is 1.82870404968e-05
Number of lifts is 24977
NMSE error is 9.94697392557e-05
Number of lifts is 25013
NMSE error is 3.55968828879e-05
Number of lifts is 22441
NMSE error is 4.64813879575e-05
Number of lifts is 22137
NMSE error is 0.000356869181231
Number of lifts is 23635
NMSE error is 0.000194714817991
Number 

KeyboardInterrupt: 

In [133]:
#G1 = nx.erdos_renyi_graph(100,0.5)
# n=50, N=4: 25sec for brute force
# n=100, N=4: 7min40sec for brute force

G3 = graphlet_list(5)[5][13]

#G2 = nx.read_edgelist('facebook/107.edges',create_using = nx.Graph()) 
#N=3, Brute force: {0: 1238448, 1: 420329} in 24s, ratios:{0:0.74684421112, 1: 0.25315578887}
#N=4, brute force in 8h48m {0: 18978657, 1: 37691180, 2: 38061593, 3: 2435907, 4: 14544473, 5: 4876123}
#Ratios: {0: 0.16278405930740705, 1: 0.3232854295478418, 2: 0.326462542225532, 
#3: 0.02089330291154574, 4: 0.12475110095656297, 5: 0.04182356505111039}


#G4 = nx.read_edgelist('com-youtube.ungraph.txt',create_using = nx.Graph())
#N=3: brute force would take 46 days

#print(G1.number_of_nodes(), G1.number_of_edges())
#print(G2.number_of_nodes(), G2.number_of_edges())

liftSRW(G2, N=3, running_time=6000, epoch_num=1, time_step=60)

For youtube graph, with random walk between lifts = number of nodes^0.6, N=4:
Time is 300 Type counter is {0: 7508388284920, 1: 319814412047, 2: 8970481104, 3: 170730656, 4: 255039295, 5: 1685807}
Number of graphlets sampled is 171514

For youtube graph, with random walk between lifts = number of nodes^0.7, N=4:
Time is 1292 Type counter is {0: 8505370717530, 1: 145268261932, 2: 29827875714, 3: 320134434, 4: 222040756, 5: 7118308}
Number of graphlets sampled is 360072

For youtube graph, with random walk between lifts = number of nodes^0.7, N=3:
Time is 301 Type counter is {0: 2217870318, 1: 2950872}
Number of graphlets sampled is 158304
Time is 648 Type counter is {0: 2005371963, 1: 3396963}
Number of graphlets sampled is 316963
Time is 1461 Type counter is {0: 1527864280, 1: 2972422}
Number of graphlets sampled is 591626

For youtube graph, with random walk between lifts = number of nodes^0.6, N=3:
Time is 309 Type counter is {0: 1781470713, 1: 3094036}
Number of graphlets sampled is 242325
Time is 648 Type counter is {0: 1452978573, 1: 2740937}
Number of graphlets sampled is 488842
Time is 900 Type counter is {0: 1552455355, 1: 2755276}
Number of graphlets sampled is 677982
Time is 1120 Type counter is {0: 1375660415, 1: 2752554}
Number of graphlets sampled is 879448
Time is 1180 Type counter is {0: 1327184466, 1: 2690900}
Number of graphlets sampled is 928773